# Stage 13 Homework Starter — Productization

## Objective
Deploy your trained model as a **reusable, handoff-ready API or dashboard** and finalize your project for reproducibility and clarity.

## Steps
1. Create a mock, very basic analysis in a notebook.
2. Clean your notebook by removing exploratory cells and documenting your code.
3. Move reusable functions into `/src/`.
4. Load your trained model from Stage 12 or earlier stages.
5. Pickle/save the model and test reload.
6. Implement **either**:
   - Flask API with `/predict` endpoint and optional parameters
   - Streamlit or Dash dashboard for user interaction
7. Include:
   - Error handling for invalid inputs
   - `requirements.txt` for reproducibility
   - Documentation in `README.md`
8. Test your deployment locally and provide evidence.
9. Organize project folders and finalize notebooks for handoff.

## 1. Create mock, very basic analysis

Download Required Dependencies

In [37]:
!pip install flask streamlit

Importing Necessary Libraries

In [38]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import joblib
import os


Data Acquisition and ingestion

In [39]:

# Generate synthetic data
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=50)



In [40]:
# Create folders if they don't exist
os.makedirs('model', exist_ok=True)

# Train a basic model
model = LinearRegression()
model.fit(X, y)



# Test that it works
print("Test prediction:", model.predict([[0.1, 0.2]]))

Test prediction: [24.63962033]


## 2. Notebook Cleanup
Remove exploratory cells and document your code.

In [41]:
# TODO: Remove exploratory cells
# TODO: Document your code clearly
# Example placeholder for cleaned analysis
print("Notebook cleaned and ready for handoff.")

Notebook cleaned and ready for handoff.


## 3. Move reusable functions to /src/
Create src/utils.py and store functions there.

Code Modularization

In [45]:
# Ensure /src/ exists
import os
# os.chdir("../")  # Change to the parent directory if needed
print("Current working directory:", os.getcwd())
os.makedirs('/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/src', exist_ok=True)

# Write a simple utility function to /src/utils.py
utils_code = '''
def calculate_metrics(X, y):
    """
    Simple function to calculate regression metrics.
    Returns dict with mean and sum of outputs.
    """
    return {"mean_y": sum(y)/len(y), "sum_y": sum(y)}
'''

with open('/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/src/utils.py', 'w') as f:
    f.write(utils_code)

print("Created /src/utils.py with calculate_metrics function.")

# Import and use the function
from src.utils import calculate_metrics

metrics = calculate_metrics([row[0] for row in X], y)
print("Metrics calculated from utility function:", metrics)

Current working directory: /Users/aditya/Documents/bootcamp_aditya_shah/homework
Created /src/utils.py with calculate_metrics function.
Metrics calculated from utility function: {'mean_y': np.float64(-14.252347300728355), 'sum_y': np.float64(-1425.2347300728354)}


## 4. Folder Structure Reminder

Ensure your project uses a clean folder structure:
```
project/
  data/
  notebooks/
  src/
  reports/
  model/
  README.md
```
For API/Dashboard: minimal example:
```
project/
    app.py
    model.pkl
    requirements.txt
    README.md
```

## 5. Pickle / Save Final Model

### TODO: Replace this with your trained model

Model Persistence

In [12]:
# Save the model to disk
joblib.dump(model, '/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/model/model.pkl')

# Load the model
loaded_model = joblib.load('/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/model/model.pkl')
print("Reloaded model prediction:", loaded_model.predict([[0.1, 0.2]]))

Reloaded model prediction: [24.63962033]


## 6. Flask API Starter

### TODO: Implement Flask endpoints for /predict and /plot

In [46]:
flask_code = '''from flask import Flask, request, jsonify, render_template_string
import joblib
import matplotlib.pyplot as plt
import io, base64

# ---- Load your model ----
model = joblib.load('model/model.pkl')
app = Flask(__name__)

# ---- Helper function ----
def predict(features):
    pred = model.predict([features])[0]
    return pred

# ---- Home Page ----
@app.route('/')
def home():
    html = """
    <html>
        <body style="font-family: Arial; padding:20px;">
            <h1>Welcome to the Flask Model API</h1>
            <p>Try these routes:</p>
            <ul>
                <li><a href="/predict/5">/predict/5</a> → Predict with one input</li>
                <li><a href="/predict/5/10">/predict/5/10</a> → Predict with two inputs</li>
                <li><a href="/plot">/plot</a> → Show demo plot</li>
            </ul>
        </body>
    </html>
    """
    return render_template_string(html)

# ---- JSON Prediction (POST) ----
@app.route('/predict', methods=['POST'])
def predict_default():
    data = request.get_json()
    features = data.get('features')
    return jsonify({'prediction': float(predict(features))})

# ---- Path-based Prediction ----

@app.route('/predict/<string:input1>')
@app.route('/predict/<string:input1>/<string:input2>')
def predict_path(input1, input2="none"):
    try:
        # convert inputs
        features = [float(input1.strip())]
        if input2 and input2.lower() != "none":
            features.append(float(input2.strip()))
    except Exception as e:
        return jsonify({'error': f'Conversion failed: {input1}, {input2}. Error: {str(e)}'}), 400

    try:
        # call model
        result = float(predict(features))
        return jsonify({'prediction': result})
    except Exception as e:
        return jsonify({'error': f'Model prediction failed for features={features}. Error: {str(e)}'}), 400


# ---- Plot Route ----
@app.route('/plot')
def plot_page():
    fig, ax = plt.subplots()
    ax.plot([0, 1, 2, 3], [10, 20, 15, 30])
    ax.set_title("Demo Plot")
    
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    
    html = f"""
    <html>
        <body style="font-family: Arial; padding:20px;">
            <h1>Model Output Plot</h1>
            <img src="data:image/png;base64,{img_base64}">
            <p><a href='/'>Back Home</a></p>
        </body>
    </html>
    """
    return render_template_string(html)

# ---- Run the app ----
if __name__ == '__main__':
    app.run(debug=True, port=5000)

'''

with open('/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/flask_streamlit/app.py','w') as f:
    f.write(flask_code)

print("Flask app saved to app.py")

Flask app saved to app.py


In [30]:
import os
# Windows command to launch Flask in separate terminal window
os.system("python app.py")
print("Flask server launched in external terminal window.")

 * Serving Flask app 'app'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 710-316-010
127.0.0.1 - - [28/Aug/2025 13:40:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2025 13:40:07] "GET /predict/7.0 HTTP/1.1" 400 -
127.0.0.1 - - [28/Aug/2025 13:40:07] "GET /predict/3.0/5.0 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2025 13:40:07] "GET /plot HTTP/1.1" 200 -


Flask server launched in external terminal window.


## 7. Testing the Flask API from Notebook

### TODO: Modify examples with your actual features

In [ ]:

import requests
from IPython.display import display, HTML

# POST /predict
response = requests.post(
    'http://127.0.0.1:5000/predict',
    json={'features':[0.1, 0.2]}
)
print(response.json())

# GET /predict/<input1>
response2 = requests.get('http://127.0.0.1:5000/predict/7.0')
print(response2.json())

# GET /predict/<input1>/<input2>
response3 = requests.get('http://127.0.0.1:5000/predict/3.0/5.0')
print(response3.json())

# GET /plot
response_plot = requests.get('http://127.0.0.1:5000/plot')
display(HTML(response_plot.text))

#Output Generated for the above requests:(Ran code in another ipynb as flask server needs to be running)

# {'prediction': 24.639620327013382}
# {'error': 'Model prediction failed for features=[7.0]. Error: X has 1 features, but LinearRegression is expecting 2 features as input.'}
# {'prediction': 643.3211162805902}

## 8. Optional Streamlit / Dash Dashboard

### TODO: Add dashboard in a separate file (`app_streamlit.py` or `app_dash.py`)

In [54]:
streamlit_code = '''
import streamlit as st
import joblib

model = joblib.load('/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/model/model.pkl')

st.title("Model Prediction Dashboard")

input_1 = st.number_input("Feature 1")
input_2 = st.number_input("Feature 2")

if st.button("Predict"):
    pred = model.predict([[input_1, input_2]])
    st.write(f"Prediction: {pred[0]}")
'''

with open('/Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/flask_streamlit/app_streamlit.py','w') as f:
    f.write(streamlit_code)

print("Streamlit app saved to app_streamlit.py")


Streamlit app saved to app_streamlit.py


In [59]:

# Launch Streamlit in external terminal (Windows example)
os.system("streamlit run /Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/flask_streamlit/app_streamlit.py")
print("Streamlit dashboard launched in external terminal window.")


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.182:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
  Stopping...
Streamlit dashboard launched in external terminal window.


## 9. Handoff Best Practices

- Ensure README.md is complete and clear
- Provide `requirements.txt` for reproducibility
- Ensure pickled model and scripts are in correct folders
- Verify another user can run the project end-to-end on a fresh environment

Generate requirements.txt

In [58]:
!pip freeze > /Users/aditya/Documents/bootcamp_aditya_shah/homework/stage13_productization/requirements.txt


In [35]:
print('Example folder structure for productization:')
print('project_root/')
print('  /data/       # raw and processed data')
print('  /src/        # reusable functions and scripts')
print('  /notebooks/  # exploration and final notebooks')
print('  /reports/    # stakeholder summaries, charts, PDFs')
print('  /model/      # pickled model objects')
print('  README.md    # project overview, assumptions, instructions')

print('Ways to productize a project:')
print('- Expose model via an API (Flask, FastAPI)')
print('- Provide an interactive dashboard (Streamlit, Dash)')
print('- Generate automated batch scripts')
print('- Create clear documentation and README for handoff')
print('- Package code and outputs for reproducibility')
print('  - Code is modularized into stage driven libraries')
print('  - Functions are flexible and allow for many possibilities')
print('  - Analysis code runs from start to finish')
print('    - Code throws errors when stuck at a point that it needs human intervention')
print('    - Code logs and/or collects all relavant information as it runs')
print('  - Analyses can be run with a variety of data sets and inputs from start to finish')
print('    - Interaction and clear documentation is given to a user for this functionality')

Example folder structure for productization:
project_root/
  /data/       # raw and processed data
  /src/        # reusable functions and scripts
  /notebooks/  # exploration and final notebooks
  /reports/    # stakeholder summaries, charts, PDFs
  /model/      # pickled model objects
  README.md    # project overview, assumptions, instructions
Ways to productize a project:
- Expose model via an API (Flask, FastAPI)
- Provide an interactive dashboard (Streamlit, Dash)
- Generate automated batch scripts
- Create clear documentation and README for handoff
- Package code and outputs for reproducibility
  - Code is modularized into stage driven libraries
  - Functions are flexible and allow for many possibilities
  - Analysis code runs from start to finish
    - Code throws errors when stuck at a point that it needs human intervention
    - Code logs and/or collects all relavant information as it runs
  - Analyses can be run with a variety of data sets and inputs from start to finish
   